In [1]:
!pip install hvplot

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 5.4 MB/s eta 0:00:00


## Bibliotecas Utilizadas

In [2]:
# ------------------------------
# Imports
# ------------------------------
import pandas as pd
import panel as pn
import hvplot.pandas  # integra Pandas -> hvPlot

In [3]:
pn.extension()

# ------------------------------
# 1. Ler a planilha
# ------------------------------
caminho = "/kaggle/input/consolidadorma/CONSOLIDADO 2024 JAN-DEZ.xlsx"
aba = "CRAS GERAL"

# Lendo sem cabeçalho
df_raw = pd.read_excel(caminho, sheet_name=aba, header=None)

# ------------------------------
# 2. Definir meses
# ------------------------------
meses = ["Janeiro", "Fevereiro", "Março", "Abril", "Maio", "Junho",
         "Julho", "Agosto", "Setembro", "Outubro", "Novembro", "Dezembro"]

col_inicio = 39  # coluna Janeiro
col_fim = 50     # coluna Dezembro

# ------------------------------
# Extrair dados da seção Recepção (1.1)
# ------------------------------
dados_recepcao = df_raw.iloc[13:16, col_inicio:col_fim+1]
#index considerado muito grande preferi colocar o indice manualmente nesse caso
dados_recepcao.index = ["Total", "Individual", "Coletivo"]
# Ajustar índices e colunas
dados_recepcao.columns = meses

# ------------------------------
# Extrair dados da seção Recepção (1.2)
# ------------------------------

dados_recepcao_horarios = df_raw.iloc[17:22, col_inicio: col_fim +1]
# Pegar os nomes das demandas na coluna A (coluna 0)
dados_recepcao_horarios_lista = df_raw.iloc[17:22, 0].tolist()
dados_recepcao_horarios.index = dados_recepcao_horarios_lista
dados_recepcao_horarios.columns = meses

# ------------------------------
# Extrair dados da seção Recepção (1.3)
# ------------------------------

# ------------------------------
# Procedência - Preparação dos Dados
# ------------------------------

# Selecionar as linhas da seção "Procedência"
procedencia = df_raw.iloc[23:44, col_inicio:col_fim+1]

# Lista dos tipos de procedência
tipos_procedencia = df_raw.iloc[23:44, 0].tolist()

# Definir índice
procedencia.index = tipos_procedencia[:len(procedencia)]

# Somar totais
totais = procedencia.sum(axis=1)

# Selecionar top 5
top_procedencia = (
    totais.sort_values(ascending=False)
    .head(5)
    .reset_index()  # transforma o índice em coluna
)

# Renomear colunas
top_procedencia.columns = ["Procedência", "Total"]

# ---- DEMANDAS (1.4) ----
# Selecionar as linhas da seção 1.4 (exemplo: 83 até 120 - ajuste conforme sua planilha)
demandas = df_raw.iloc[45:76, col_inicio:col_fim+1]

# Pegar os nomes das demandas na coluna A (coluna 0)
nomes_demandas = df_raw.iloc[45:76, 0].tolist()

# Definir colunas e índices
demandas.columns = meses
demandas.index = nomes_demandas

# Somar o total por demanda ao longo do ano
totais_demandas = demandas.sum(axis=1)

# Selecionar top 5
top_demandas = (
    totais.sort_values(ascending=False)
    .head(10)
    .reset_index()  # transforma o índice em coluna
)

# Renomear colunas
top_demandas.columns = ["Demandas", "Total"]

# ------------------------------
# Extrair dados da seção Recepção (1.6)
# ------------------------------

dados_recepcao_tipo_atend = df_raw.iloc[81:84, col_inicio: col_fim +1]
# Pegar os nomes das demandas na coluna A (coluna 0)
dados_recepcao_tipo_atend_lista = df_raw.iloc[81:84, 0].tolist()
dados_recepcao_tipo_atend.index = dados_recepcao_tipo_atend_lista
dados_recepcao_tipo_atend.columns = meses

# ------------------------------
# Extrair dados da seção Recepção (1.7)
# ------------------------------

dados_recepcao_nacionalidade = df_raw.iloc[85:92, col_inicio:col_fim+1]
# Pegar os nomes das demandas na coluna A (coluna 0)
dados_recepcao_nacionalidade_lista = df_raw.iloc[85:92, 0].tolist()
# Ajustar índices e colunas
dados_recepcao_nacionalidade.index = dados_recepcao_nacionalidade_lista
dados_recepcao_nacionalidade.columns = meses
dados_recepcao_nacionalidade_sembrasileiros = dados_recepcao_nacionalidade.drop("Brasileiro", errors="ignore")

# -----------------------------------------
# Recepção por Idioma (Total) - Correção Final e Segura
# -----------------------------------------

# Selecionar os dados brutos do trecho correspondente
dados_recepcao_idioma = df_raw.iloc[93:98, col_inicio:col_fim+1].copy()

# Pegar os nomes dos idiomas (coluna A)
dados_recepcao_idioma.index = df_raw.iloc[93:98, 0].values

# Remover o idioma "Português" (não relevante para análise)
dados_recepcao_idioma = dados_recepcao_idioma.drop("Português", errors="ignore")

# Somar os valores de cada idioma (somando todas as colunas/mês)
dados_recepcao_idioma_total = pd.DataFrame({
    "Idioma": dados_recepcao_idioma.index,
    "Total": dados_recepcao_idioma.sum(axis=1).values
})



In [4]:
# -----------------------------------------
# Recepção por Idioma (Total) - Correção Final e Segura
# -----------------------------------------

# Selecionar os dados brutos do trecho correspondente
dados_recepcao_deficiencia = df_raw.iloc[99:107, col_inicio:col_fim+1].copy()

# Pegar os nomes dos idiomas (coluna A)
dados_recepcao_deficiencia.index = df_raw.iloc[99:107, 0].values

# Remover o idioma "Português" (não relevante para análise)
dados_recepcao_deficiencia = dados_recepcao_deficiencia.drop("Sem deficiência", errors="ignore")

# Somar os valores de cada idioma (somando todas as colunas/mês)
dados_recepcao_deficiencia_total = pd.DataFrame({
    "Deficiência": dados_recepcao_deficiencia.index,
    "Total": dados_recepcao_deficiencia.sum(axis=1).values
})

# Abas que queremos comparar
abas = [
    "CRAS FALCHI",
    "CRAS FEITAL",
    "CRAS MACUCO",
    "CRAS ORATORIO",
    "CRAS PARQUE",
    "CRAS SAO JOAO",
    "CRAS VILA",
    "CRAS ZAIRA"
]

# Linha onde está o total de famílias cadastradas
linha_familias = 131

# Dicionário para armazenar os resultados
dados_familias = {}

# Loop pelas abas
for aba in abas:
    # Ler planilha
    df_raw = pd.read_excel(caminho, sheet_name=aba, header=None)
    
    # Extrair os valores da linha desejada
    valores = df_raw.iloc[linha_familias, col_inicio:col_fim+1].tolist()
    
    # Guardar no dicionário
    dados_familias[aba.replace("CRAS ", "")] = valores

# Converter em DataFrame
df_familias = pd.DataFrame(dados_familias, index=meses).T



In [5]:
# Exibir resultado
df_familias

,Janeiro,Fevereiro,Março,Abril,Maio,Junho,Julho,Agosto,Setembro,Outubro,Novembro,Dezembro
FALCHI,1407,1414,1420,1431,1436,1443,1450,1460,1469,1478,1479,1493
FEITAL,6770,6774,6662,6480,6197,5786,5671,5548,5598,5663,5696,5715
MACUCO,4221,4295,4422,4471,4508,4555,5283,5316,5326,5384,5392,5403
ORATORIO,9456,9468,9478,9500,9510,9521,9547,9602,9615,9584,9613,9613
PARQUE,4131,4136,4144,4155,4174,4188,4191,4215,3011,3023,3050,3055
SAO JOAO,9815,5225,5249,5332,5415,5415,5451,5511,5562,5562,5511,5571
VILA,3387,3164,2919,2938,2959,2974,2988,3005,3025,3039,2952,2963
ZAIRA,4112,4126,4136,4145,4151,4159,4165,4182,4192,4192,4233,4239


## criando os Gráficos

In [6]:
# ------------------------------
# 4. Criar gráfico interativo com hvPlot
# ------------------------------
grafico_recepcao = dados_recepcao.T.hvplot.bar(
    height=500,
    width=1200,
    title="Recepção - Quantidade de Pessoas - Cras Geral (2024)",
    xlabel="Meses",
    ylabel="Quantidade de Pessoas",
    rot=50,        # rotação dos meses
    tools=["hover"],
    stacked=False  # barras lado a lado
)

grafico_recepca_horarios = dados_recepcao_horarios.T.hvplot.line(
    height=500,
    width=1200,
    title="Atendimentos por Horário - CRAS Geral 2024",
    xlabel="Meses",
    ylabel="Número de Atendimentos",
    rot=45,        # rotação dos meses
    tools=["hover"],
    stacked=False  # barras lado a lado
)

# ------------------------------
# Procedência - Gráfico Interativo
# ------------------------------

grafico_procedencia = top_procedencia.hvplot.bar(
    x="Procedência",
    y="Total",
    height=500,
    width=1200,
    title="Procedência - Top 5 Atendimentos - CRAS Geral (2024)",
    xlabel="Procedência",
    ylabel="Número de Atendimentos",
    rot=45,
    tools=["hover"],
    color="skyblue",
    line_color="black"
)

grafico_demandas = top_demandas.hvplot.barh(
    x="Demandas",
    y="Total",
    height=500,
    width=1200,
    title="Top 10 Demandas - CRAS Geral (2024)",
    xlabel="Demandas",
    ylabel="Número de Atendimentos",
    rot=45,
    tools=["hover"],
    color="skyblue",
    line_color="black"
)

# ------------------------------
# Tipo de Atendimento - Gráfico Interativo
# ------------------------------
grafico_recepcao_tipo_atend = dados_recepcao_tipo_atend.T.hvplot.bar(
    height=500,
    width=1200,
    title="Recepção por Tipo de Atendimento - CRAS Geral (2024)",
    xlabel="Meses",
    ylabel="Número de Atendimentos",
    rot=45,
    tools=["hover"],
    stacked=True  # empilha para ver o total por mês
)

grafico_recepcao_nacionalidade = dados_recepcao_nacionalidade_sembrasileiros.T.hvplot.bar(
    height=500,
    width=1200,
    title="Recepção por Nacionalidade (Exceto Brasileiros) - CRAS Geral 2024",
    xlabel="Meses",
    ylabel="Número de Atendimentos",
    rot=45,
    tools=["hover"],
    stacked=False,
    ylim=(0, dados_recepcao_nacionalidade_sembrasileiros.values.max() + 10),
    shared_axes=False  # <<< força eixo X e Y independentes
)

# Gráfico com ajustes visuais
grafico_recepcao_idioma_total = dados_recepcao_idioma_total.hvplot.bar(
    x="Idioma",
    y="Total",
    height=500,
    width=1200,
    title="Total de Atendimentos por Idioma - CRAS Geral (2024)",
    xlabel="Idioma",
    ylabel="Número de Atendimentos",
    rot=45,
    tools=["hover"],
    color="skyblue",
    line_color="black",
    ylim=(0, dados_recepcao_idioma_total["Total"].max() + 5),  # escala justa
    shared_axes=False,
    responsive=True,
    bar_width=0.5  # controla a espessura das barras
)

grafico_recepcao_deficiencia_total = dados_recepcao_deficiencia_total.hvplot.bar(
    x="Deficiência",
    y="Total",
    height=500,
    width=1200,
    title="Total de Atendimentos por Deficiência - CRAS Geral (2024)",
    xlabel="Deficiência",
    ylabel="Número de Atendimentos",
    rot=45,
    tools=["hover"],
    color="skyblue",
    line_color="black",
    ylim=(0, dados_recepcao_deficiencia_total["Total"].max() + 5),  # escala justa
    shared_axes=False,
    responsive=True,
    bar_width=0.5  # controla a espessura das barras
)

grafico_familias = df_familias.T.hvplot.bar(
    stacked=True,                # barras empilhadas
    height=500,
    width=1200,
    title="Total de Famílias Cadastradas por CRAS (2024)",
    xlabel="Meses",
    ylabel="Número de Famílias",
    rot=45,
    tools=["hover"],
    legend="top_left",
    cmap="Set3",# paleta neutra e suave
    shared_axes=False
    
)



In [7]:
# ------------------------------
# 5. Criar layout do painel
# ------------------------------
layout = pn.Column(
    "# Dashboard Socioassistencial RMA",
    "Exemplo: Recepção - Quantidade de Pessoas",
    dados_recepcao,  # tabela
    grafico_recepcao,                                   # gráfico interativo
    grafico_recepca_horarios,
    grafico_procedencia,
    grafico_demandas,
    grafico_recepcao_tipo_atend,
    dados_recepcao_nacionalidade,
    "Para analisar a diversidade de estrangeiros optamos por excluir a linha com brasileiros",
    grafico_recepcao_nacionalidade,
    grafico_recepcao_idioma_total,
    grafico_recepcao_deficiencia_total,
    grafico_familias
       
)

layout

Column
    [0] Markdown(str)
    [1] Markdown(str)
    [2] DataFrame(DataFrame)
    [3] HoloViews(Bars, height=500, sizing_mode='fixed', width=1200)
    [4] HoloViews(NdOverlay, height=500, sizing_mode='fixed', width=1200)
    [5] HoloViews(Bars, height=500, sizing_mode='fixed', width=1200)
    [6] HoloViews(Bars, height=500, sizing_mode='fixed', width=1200)
    [7] HoloViews(Bars, height=500, sizing_mode='fixed', width=1200)
    [8] DataFrame(DataFrame)
    [9] Markdown(str)
    [10] HoloViews(Bars, height=500, sizing_mode='fixed', width=1200)
    [11] HoloViews(Bars, height=500, sizing_mode='fixed', width=1200)
    [12] HoloViews(Bars, height=500, sizing_mode='fixed', width=1200)
    [13] HoloViews(Bars, height=500, sizing_mode='fixed', width=1200)